In [ ]:
!pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=f8822cbffa18fcaad4007b0dd1fdb92bbad44ef2d7d87d587c9b35cfae693fa8
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

# ==================== Load the Dataset ==================== #
def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    df.dropna(inplace=True)
    return df

# ==================== Define Strategy ==================== #
class DualMA_Strategy(Strategy):
    fast_ma = 10
    slow_ma = 50
    stop_loss_pct = 0.02
    take_profit_pct = 0.04

    def init(self):
        price = self.data.Close
        self.sma_fast = self.I(SMA, price, self.fast_ma)
        self.sma_slow = self.I(SMA, price, self.slow_ma)

    def next(self):
        price = self.data.Close[-1]
        if crossover(self.sma_fast, self.sma_slow):
            self.buy(sl=price * (1 - self.stop_loss_pct), tp=price * (1 + self.take_profit_pct))
        elif crossover(self.sma_slow, self.sma_fast):
            self.position.close()

# ==================== Backtest for a Single Dataset ==================== #
def run_backtest_single_dataset(file_path):
    df = load_data(file_path)
    bt = Backtest(df, DualMA_Strategy, cash=100000, commission=0.001, exclusive_orders=True)

    stats = bt.run()
    trades = stats['_trades']  # Extract trade data

    # Prepare trade details
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    trade_details = trades[['EntryBar', 'ExitBar', 'EntryPrice', 'ExitPrice', 'Size', 'PnL']].copy()
    trade_details['name'] = file_name
    trade_details['entry datetime'] = pd.to_datetime(df.index[trade_details['EntryBar']]).strftime('%d %b %y')
    trade_details['exit datetime'] = pd.to_datetime(df.index[trade_details['ExitBar']]).strftime('%d %b %y')
    trade_details['entry price'] = trade_details['EntryPrice']
    trade_details['exit price'] = trade_details['ExitPrice']
    trade_details['quantity'] = trade_details['Size']
    trade_details['profit/loss value'] = trade_details['PnL']

    return trade_details[['name', 'entry datetime', 'entry price', 'exit datetime', 'exit price', 'quantity', 'profit/loss value']]

# ==================== Process Multiple Files ==================== #
def process_all_files_in_zip(zip_file_path, output_csv):
    extracted_folder = "extracted_files"
    os.makedirs(extracted_folder, exist_ok=True)

    # Step 1: Extract ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder)
    print("ZIP file extracted successfully.")

    # Step 2: Iterate through all CSV files
    all_results = pd.DataFrame()
    for root, _, files in os.walk(extracted_folder):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file}")
                try:
                    trade_results = run_backtest_single_dataset(file_path)
                    all_results = pd.concat([all_results, trade_results], ignore_index=True)
                except Exception as e:
                    print(f"Error processing {file}: {e}")

    # Step 3: Save all results to a single CSV
    all_results.to_csv(output_csv, index=False)
    print(f"All trade results saved to: {output_csv}")

# ==================== Main Execution ==================== #
if __name__ == "__main__":
    # Path to the uploaded ZIP file in Google Colab
    zip_file_path = "/content/drive-download-20241217T111005Z-001.zip"  # Replace with your uploaded ZIP file path

    # Output file path
    output_csv = "/content/all_trade_results.csv"

    # Process all files
    process_all_files_in_zip(zip_file_path, output_csv)


ZIP file extracted successfully.
Processing file: Punjab National Bank.csv
Processing file: Axis Bank.csv
Processing file: Power Finance Corporation.csv
Processing file: Adani Power.csv
Processing file: Rural Electrification Corporation.csv
Processing file: Larsen & Toubro.csv
Processing file: Hindalco Industries.csv
Processing file: Vedanta.csv
Processing file: DLF.csv
Processing file: Adani Enterprises.csv
Processing file: Tata Steel.csv
Processing file: ICICI Bank.csv
Processing file: Adani Ports & SEZ.csv
Processing file: Tata Motors.csv
Processing file: State Bank of India.csv
All trade results saved to: /content/all_trade_results.csv


In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

# ==================== Load the Dataset ==================== #
def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    df.dropna(inplace=True)
    return df

# ==================== Define Strategy ==================== #
class DualMA_Strategy(Strategy):
    fast_ma = 10
    slow_ma = 50
    stop_loss_pct = 0.02
    take_profit_pct = 0.04

    def init(self):
        price = self.data.Close
        self.sma_fast = self.I(SMA, price, self.fast_ma)
        self.sma_slow = self.I(SMA, price, self.slow_ma)

    def next(self):
        price = self.data.Close[-1]
        if crossover(self.sma_fast, self.sma_slow):
            self.buy(sl=price * (1 - self.stop_loss_pct), tp=price * (1 + self.take_profit_pct))
        elif crossover(self.sma_slow, self.sma_fast):
            self.position.close()

# ==================== Backtest for a Single Dataset ==================== #
def run_backtest_single_dataset(file_path):
    df = load_data(file_path)
    bt = Backtest(df, DualMA_Strategy, cash=100000, commission=0.001, exclusive_orders=True)

    stats = bt.run()
    trades = stats['_trades']  # Extract trade data

    # Prepare trade details
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    trade_details = trades[['EntryBar', 'ExitBar', 'EntryPrice', 'ExitPrice', 'Size', 'PnL']].copy()
    trade_details['name'] = file_name
    trade_details['entry datetime'] = pd.to_datetime(df.index[trade_details['EntryBar']]).strftime('%d %b %y')
    trade_details['exit datetime'] = pd.to_datetime(df.index[trade_details['ExitBar']]).strftime('%d %b %y')
    trade_details['entry price'] = trade_details['EntryPrice']
    trade_details['exit price'] = trade_details['ExitPrice']
    trade_details['quantity'] = trade_details['Size']
    trade_details['profit/loss value'] = trade_details['PnL']

    # Print Sharpe Ratio
    sharpe_ratio = stats['Sharpe Ratio']
    print(f"File: {file_name} | Sharpe Ratio: {sharpe_ratio:.2f}")

    return trade_details[['name', 'entry datetime', 'entry price', 'exit datetime', 'exit price', 'quantity', 'profit/loss value']]

# ==================== Process Multiple Files ==================== #
def process_all_files_in_zip(zip_file_path, output_csv):
    extracted_folder = "extracted_files"
    os.makedirs(extracted_folder, exist_ok=True)

    # Step 1: Extract ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder)
    print("ZIP file extracted successfully.")

    # Step 2: Iterate through all CSV files
    all_results = pd.DataFrame()
    for root, _, files in os.walk(extracted_folder):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file}")
                try:
                    trade_results = run_backtest_single_dataset(file_path)
                    all_results = pd.concat([all_results, trade_results], ignore_index=True)
                except Exception as e:
                    print(f"Error processing {file}: {e}")

    # Step 3: Save all results to a single CSV
    all_results.to_csv(output_csv, index=False)
    print(f"All trade results saved to: {output_csv}")

# ==================== Main Execution ==================== #
if __name__ == "__main__":
    # Path to the uploaded ZIP file in Google Colab
    zip_file_path = "/content/drive-download-20241217T111005Z-001.zip"  # Replace with your uploaded ZIP file path

    # Output file path
    output_csv = "/content/all_trade_results.csv"

    # Process all files
    process_all_files_in_zip(zip_file_path, output_csv)


ZIP file extracted successfully.
Processing file: Punjab National Bank.csv
File: Punjab National Bank | Sharpe Ratio: 0.59
Processing file: Axis Bank.csv
File: Axis Bank | Sharpe Ratio: 0.10
Processing file: Power Finance Corporation.csv
File: Power Finance Corporation | Sharpe Ratio: 0.01
Processing file: Adani Power.csv
File: Adani Power | Sharpe Ratio: 0.87
Processing file: Rural Electrification Corporation.csv
File: Rural Electrification Corporation | Sharpe Ratio: 0.00
Processing file: Larsen & Toubro.csv
File: Larsen & Toubro | Sharpe Ratio: 0.00
Processing file: Hindalco Industries.csv
File: Hindalco Industries | Sharpe Ratio: 0.48
Processing file: Vedanta.csv
File: Vedanta | Sharpe Ratio: 0.56
Processing file: DLF.csv
File: DLF | Sharpe Ratio: 0.36
Processing file: Adani Enterprises.csv
File: Adani Enterprises | Sharpe Ratio: 0.00
Processing file: Tata Steel.csv
File: Tata Steel | Sharpe Ratio: 0.61
Processing file: ICICI Bank.csv
File: ICICI Bank | Sharpe Ratio: 0.21
Processin

In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

# ==================== Load the Dataset ==================== #
def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    df.dropna(inplace=True)
    return df

# ==================== Define Strategy ==================== #
class DualMA_Strategy(Strategy):
    fast_ma = 10
    slow_ma = 50
    stop_loss_pct = 0.02
    take_profit_pct = 0.04

    def init(self):
        price = self.data.Close
        self.sma_fast = self.I(SMA, price, self.fast_ma)
        self.sma_slow = self.I(SMA, price, self.slow_ma)

    def next(self):
        price = self.data.Close[-1]
        if crossover(self.sma_fast, self.sma_slow):
            self.buy(sl=price * (1 - self.stop_loss_pct), tp=price * (1 + self.take_profit_pct))
        elif crossover(self.sma_slow, self.sma_fast):
            self.position.close()

# ==================== Optimized Backtest for a Single Dataset ==================== #
def run_optimized_backtest(file_path):
    df = load_data(file_path)
    bt = Backtest(df, DualMA_Strategy, cash=100000, commission=0.001, exclusive_orders=True)

    # Optimize with limited parameters and early stopping
    stats = bt.optimize(
        fast_ma=range(5, 30, 5),   # Reduced fast MA range
        slow_ma=range(20, 100, 10), # Reduced slow MA range
        stop_loss_pct=[0.01, 0.02, 0.03],  # Limited stop loss percentages
        take_profit_pct=[0.03, 0.05, 0.07],  # Limited take profit percentages
        maximize='Sharpe Ratio',
        constraint=lambda p: p.fast_ma < p.slow_ma,
        max_tries=50  # Limit optimization tries
    )

    # Check if Sharpe Ratio is greater than 1
    sharpe_ratio = stats['Sharpe Ratio']
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    if sharpe_ratio > 1:
        print(f"File: {file_name} | Sharpe Ratio: {sharpe_ratio:.2f}")

        # Extract trade details
        trades = stats['_trades']
        trade_details = trades[['EntryBar', 'ExitBar', 'EntryPrice', 'ExitPrice', 'Size', 'PnL']].copy()
        trade_details['name'] = file_name
        trade_details['entry datetime'] = pd.to_datetime(df.index[trade_details['EntryBar']]).strftime('%d %b %y')
        trade_details['exit datetime'] = pd.to_datetime(df.index[trade_details['ExitBar']]).strftime('%d %b %y')
        trade_details['entry price'] = trade_details['EntryPrice']
        trade_details['exit price'] = trade_details['ExitPrice']
        trade_details['quantity'] = trade_details['Size']
        trade_details['profit/loss value'] = trade_details['PnL']

        return trade_details[['name', 'entry datetime', 'entry price', 'exit datetime', 'exit price', 'quantity', 'profit/loss value']]
    else:
        print(f"File: {file_name} | Sharpe Ratio: {sharpe_ratio:.2f} (Below 1) - Skipping")
        return None

# ==================== Process Multiple Files ==================== #
def process_all_files_in_zip(zip_file_path, output_csv):
    extracted_folder = "extracted_files"
    os.makedirs(extracted_folder, exist_ok=True)

    # Step 1: Extract ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder)
    print("ZIP file extracted successfully.")

    # Step 2: Iterate through all CSV files
    all_results = pd.DataFrame()
    for root, _, files in os.walk(extracted_folder):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file}")
                try:
                    trade_results = run_optimized_backtest(file_path)
                    if trade_results is not None:
                        all_results = pd.concat([all_results, trade_results], ignore_index=True)
                except Exception as e:
                    print(f"Error processing {file}: {e}")

    # Step 3: Save all results to a single CSV
    if not all_results.empty:
        all_results.to_csv(output_csv, index=False)
        print(f"Optimized trade results saved to: {output_csv}")
    else:
        print("No files produced a Sharpe Ratio > 1. No results saved.")

# ==================== Main Execution ==================== #
if __name__ == "__main__":
    # Path to the uploaded ZIP file in Google Colab
    zip_file_path = "/content/drive-download-20241217T111005Z-001.zip"  # Replace with your uploaded ZIP file path

    # Output file path
    output_csv = "/content/optimized_trade_results.csv"

    # Process all files
    process_all_files_in_zip(zip_file_path, output_csv)


ZIP file extracted successfully.
Processing file: Punjab National Bank.csv


Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

File: Punjab National Bank | Sharpe Ratio: 1.64
Processing file: Axis Bank.csv


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

File: Axis Bank | Sharpe Ratio: 1.39
Processing file: Power Finance Corporation.csv


...

File: Adani Ports & SEZ | Sharpe Ratio: 2.00
Processing file: Tata Motors.csv


Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

File: Tata Motors | Sharpe Ratio: 1.37
Processing file: State Bank of India.csv


Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

File: State Bank of India | Sharpe Ratio: 1.15
Optimized trade results saved to: /content/optimized_trade_results.csv


In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    df.dropna(inplace=True)
    return df

class DualMA_Strategy(Strategy):
    fast_ma = 10
    slow_ma = 50
    stop_loss_pct = 0.02
    take_profit_pct = 0.04

    def init(self):
        price = self.data.Close
        self.sma_fast = self.I(SMA, price, self.fast_ma)
        self.sma_slow = self.I(SMA, price, self.slow_ma)

    def next(self):
        price = self.data.Close[-1]
        if crossover(self.sma_fast, self.sma_slow):
            self.buy(sl=price * (1 - self.stop_loss_pct), tp=price * (1 + self.take_profit_pct))
        elif crossover(self.sma_slow, self.sma_fast):
            self.position.close()

def run_optimized_backtest(file_path):
    df = load_data(file_path)
    bt = Backtest(df, DualMA_Strategy, cash=100000, commission=0.001, exclusive_orders=True)

    stats = bt.optimize(
        fast_ma=range(5, 30, 5),
        slow_ma=range(20, 100, 10),
        stop_loss_pct=[0.01, 0.02, 0.03],
        take_profit_pct=[0.03, 0.05, 0.07],
        maximize='Sharpe Ratio',
        constraint=lambda p: p.fast_ma < p.slow_ma,
        max_tries=50
    )

    sharpe_ratio = stats['Sharpe Ratio']
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    if sharpe_ratio > 1:
        print(f"File: {file_name} | Sharpe Ratio: {sharpe_ratio:.2f}")

        trades = stats['_trades']
        trade_details = trades[['EntryBar', 'ExitBar', 'EntryPrice', 'ExitPrice', 'Size', 'PnL']].copy()
        trade_details['name'] = file_name
        trade_details['entry datetime'] = pd.to_datetime(df.index[trade_details['EntryBar']]).strftime('%d %b %y')
        trade_details['exit datetime'] = pd.to_datetime(df.index[trade_details['ExitBar']]).strftime('%d %b %y')
        trade_details['entry price'] = trade_details['EntryPrice']
        trade_details['exit price'] = trade_details['ExitPrice']
        trade_details['quantity'] = trade_details['Size']
        trade_details['profit/loss value'] = trade_details['PnL']

        return trade_details[['name', 'entry datetime', 'entry price', 'exit datetime', 'exit price', 'quantity', 'profit/loss value']]
    else:
        print(f"File: {file_name} | Sharpe Ratio: {sharpe_ratio:.2f} (Below 1) - Skipping")
        return None

def process_all_files_in_zip(zip_file_path, output_csv):
    extracted_folder = "extracted_files"
    os.makedirs(extracted_folder, exist_ok=True)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder)
    print("ZIP file extracted successfully.")

    all_results = pd.DataFrame()
    for root, _, files in os.walk(extracted_folder):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file}")
                try:
                    trade_results = run_optimized_backtest(file_path)
                    if trade_results is not None:
                        all_results = pd.concat([all_results, trade_results], ignore_index=True)
                except Exception as e:
                    print(f"Error processing {file}: {e}")

    if not all_results.empty:
        all_results.to_csv(output_csv, index=False)
        print(f"Optimized trade results saved to: {output_csv}")
    else:
        print("No files produced a Sharpe Ratio > 1. No results saved.")

if __name__ == "__main__":
    zip_file_path = "/content/drive-download-20241217T111005Z-001.zip"
    output_csv = "/content/optimized_trade_results.csv"
    process_all_files_in_zip(zip_file_path, output_csv)
    #https://drive.google.com/drive/folders/1AW0tLyCwxhyFaQ8VMwNjQydbi0SLe9yP//testiglik


ZIP file extracted successfully.
Processing file: Punjab National Bank.csv


Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

File: Punjab National Bank | Sharpe Ratio: 1.88
Processing file: Axis Bank.csv


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

File: Axis Bank | Sharpe Ratio: 0.95 (Below 1) - Skipping
Processing file: Power Finance Corporation.csv


...

File: Adani Ports & SEZ | Sharpe Ratio: 1.69
Processing file: Tata Motors.csv


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

File: Tata Motors | Sharpe Ratio: 0.64 (Below 1) - Skipping
Processing file: State Bank of India.csv


Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

File: State Bank of India | Sharpe Ratio: 1.30
Optimized trade results saved to: /content/optimized_trade_results.csv
